In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
im = Image.open('/content/drive/MyDrive/Colab Notebooks/dem.tif')
im.show()
import numpy
dem = numpy.array(im)

cim = Image.open('/content/drive/MyDrive/Colab Notebooks/studyarea1.tif')
cim.show()
cb = numpy.array(cim)

lim = Image.open('/content/drive/MyDrive/Colab Notebooks/lc.tif')
lim.show()
lc = numpy.array(lim)

In [ ]:
import numpy as np

In [ ]:
lim = Image.open('/content/drive/MyDrive/Colab Notebooks/ex.tiff')
lim.show()
w_h1 = numpy.array(lim)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import math
import gdal

In [ ]:
pip install dbfread

In [ ]:
###
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import dbfread
from dbfread import DBF
import math
import gdal


path = r'/content/drive/MyDrive/Colab Notebooks/dem.dbf'
table = DBF(path, encoding = 'GBK')
df = pd.DataFrame(iter(table))
df = df.pivot(index ='X', columns = 'Y', values = 'F18TWK83_1')
df = df.values
dem = df #dem

path = r'/content/drive/MyDrive/Colab Notebooks/lulc.dbf'
table = DBF(path, encoding = 'GBK')
dl = pd.DataFrame(iter(table))
lc = dl.pivot(index='X', columns ='Y', values = 'Extract__1')
Cd = lc.values #Cd values from land cover

path = r'/content/drive/MyDrive/Colab Notebooks/cb.dbf'
table = DBF(path, encoding = 'GBK')
dc = pd.DataFrame(iter(table))
dc = dc.pivot(index='X', columns ='Y', values = 'cb_trial_t')
cb=dc.values #catchment basin 

###


ModuleNotFoundError: ignored

In [ ]:
Cd = lc
Cd = Cd.astype('float64')
Cd[Cd == 1] = 0.15
Cd[Cd == 2] = 0.16
Cd[Cd == 3] = 0.01
Cd[Cd == 4] = 0.005
Cd[Cd == 5] = 0.005
Cd[Cd == 6] = 0.005
Cd[Cd == 7] = 0.005
Cd[Cd == 8] = 0.005
Cd=np.nan_to_num(Cd)
Cd[Cd == 0] = 0.005

In [ ]:
for a in range(0, len(Cd)):
  for b in range(0, len(Cd[0])):
    if Cd[a,b] in [0.15,0.16,0.01,0.005]:
      Cd[a,b] = Cd[a,b]
    else:
      print((a,b))

KeyboardInterrupt: ignored

In [ ]:

import pandas as pd

import urllib3
urllib3.disable_warnings()
import matplotlib.pyplot as plt

from dateutil import parser
from datetime import timedelta
import matplotlib.dates as mdates
import numpy as np
from matplotlib.lines import Line2D
from IPython.core.display import display, HTML
from IPython.display import IFrame
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
display(HTML('<style>.container { width:100% !important; }</style>'))
mpl.rcParams['figure.figsize'] = [24, 12]

In [ ]:
ida = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/ida.csv')

In [ ]:
ida = np.array(ida)

In [ ]:
ida = ida[:,1]

In [ ]:
ida=np.delete(ida,0)

In [ ]:
ida_m = ida/141732

In [ ]:
ida_m

In [ ]:
def direction(dem):
    d=np.zeros((len(dem), len(dem[0]), 2))
    d_i = np.zeros(((len(dem)), len(dem[0])), 'int')
    for a in range(1,len(dem)-1):
        for b in range(1,len(dem[0])-1):
            minlist = [0,0,0,0,0,0,0,0,0]
            minlist[0] = dem[a-1,b-1]
            minlist[1] = dem[a-1, b]
            minlist[2] = dem[a-1, b+1]
            minlist[3] = dem[a, b-1]
            minlist[4] = dem[a, b]
            minlist[5] = dem[a, b+1]
            minlist[6] = dem[a+1, b-1]
            minlist[7] = dem[a+1, b]
            minlist[8] = dem[a+1, b+1]
            
            k = np.argmin(minlist)
            d_i[a,b]=k
        
            if k == 0:
                l = (a-1, b-1)
                d[a,b] = l
            if k == 1:
                l = (a-1, b)
                d[a,b] = l
            if k == 2:
                l = (a-1, b+1)
                d[a,b] = l
            if k == 3:
                l = (a, b-1)
                d[a,b] = l
            if k == 4:
                l = (a, b)
                d[a,b] = l
            if k == 5:
                l = (a, b+1)
                d[a,b] = l
            if k == 6:
                l = (a+1, b-1)
                d[a,b] = l
            if k == 7:
                l = (a+1, b)
                d[a,b] = l
            if k==8:
                l = (a+1, b+1)
                d[a,b] = l
                
            
    return d, d_i


In [ ]:
import scipy.constants
def get_velocity(r, h1, h2, l1, l2, C_D):
  g = scipy.constants.g
  u = np.sqrt(get_positive(g*(((l1-l2)/np.sqrt((l1-l2)**2 + r**2)-(h2-h1)/r))*max(h1,h2)/C_D))

  return u

In [ ]:
def get_positive(sequence):
  if sequence<0:
    sequence = 0
  else :
    sequence = sequence

  return sequence

In [ ]:
def vel (r_, d_i, Cd, d, w_h, dem, h0):

    u = np.zeros((len(dem), len(dem[0])))
    for a in range(1,len(dem)-1):
        for b in range(1,len(dem[0])-1):
            (x,y) = d[a,b]
            x=x.astype(int)
            y=y.astype(int)
            if d_i[a,b] ==4:
              u[a,b] = 0
            elif d_i[a,b] in [1,3,5,7]:
                r = r_
                h1 = w_h[a,b]
                h2 = w_h[x,y]
                l1, l2 = dem[a,b], dem[x,y]                
                u[a,b] = get_velocity(r, h1, h2, l1, l2, Cd[a,b] )
            
            elif d_i[a,b] in [0,2,6,8]:
                r = r_*1.4142
                h1 = w_h[a,b]
                h2 = w_h[x,y]
                l1, l2 = dem[a,b], dem[x,y]
                u[a,b] = get_velocity(r, h1, h2, l1, l2, Cd[a,b] )

    return u


In [ ]:
def vol(z, r, d_i, dem, w_h, c_V, u, d, cb, h0, dt):
    m_out = np.zeros((len(dem),len(dem[0])))    
    m_in = np.zeros((len(dem),len(dem[0])))
    giving_water = np.zeros((len(dem),len(dem[0])))
    for a in range(1,len(dem)-1):
        for b in range(1,len(dem[0])-1):
            (x,y) = d[a,b]
            x=x.astype(int)
            y=y.astype(int)
            giving_water[a,b]= get_positive(z[a,b]-z[x,y])            
            o = u[a,b]*1*dt*giving_water[a,b]*r
            p = (1/1.4142)*u[a,b]*dt*(r-u[a,b]*dt/1.4142)*giving_water[a,b]
            q = 1/2*(u[a,b]*dt)**2*giving_water[a,b]
            l = (1.4142*r*u[a,b]*dt-(1/2)*((u[a,b]*dt)**2))*giving_water[a,b]
            if d_i[a,b] in [1,3,5,7]: 
                m_out[a,b] = m_out[a,b] + o
                m_in[x,y] = m_in[x,y] + o

            elif d_i[a,b] ==0:
                m_out[a,b] = m_out[a,b] + l
                m_in[x,y] = m_in[x,y] + q
                m_in[a,b-1] = m_in[a,b-1] + p
                m_in[a-1,b] = m_in[a-1,b] + p
                
            elif d_i[a,b] ==2:
                m_out[a,b] = m_out[a,b] +  l
                m_in[x,y] = m_in[x,y] + q
                m_in[a,b+1] = m_in[a,b+1] + p
                m_in[a-1,b] = m_in[a-1,b] + p
                 
            elif d_i[a,b] ==6:
                m_out[a,b] = m_out[a,b] + l
                m_in[x,y] = m_in[x,y] + q
                m_in[a,b-1] = m_in[a,b-1] + p
                m_in[a+1,b] = m_in[a+1,b] + p

            elif d_i[a,b] ==8:
                m_out[a,b] = m_out[a,b] + l
                m_in[x,y] = m_in[x,y] + q
                m_in[a+1,b] = m_in[a+1,b] + p
                m_in[a,b+1] = m_in[a,b+1] + p
              
    for a in range(1,len(dem)-1):
        for b in range(1,len(dem[0])-1):
          
          w_h[a,b] = (w_h[a,b]*r**2+m_in[a,b]-m_out[a,b])/r**2
          if w_h[a,b] <=0:
            w_h[a,b] = 0   
        
    for a in range (1, len(dem)-1):
        for  b in range(1,len(dem[0])-1):
          if cb[a,b] ==1:
            c_V[a,b] = c_V[a,b] + w_h[a,b]
            w_h[a,b] = 0

          z[a,b] = w_h[a,b]+dem[a,b]

    

    return z, w_h, c_V


In [ ]:
def balance(z, w_h, dem):
    for a in range(1, len(z)-1):
      for b in range(1, len(z[0])-1):
        if z[a,b]-z[a,b-1]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a,b-1] = w_h[a,b-1] +0.1*w_h[a,b]
        if z[a,b]-z[a,b+1]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a,b+1] = w_h[a,b+1] +0.1*w_h[a,b]
        if z[a,b]-z[a+1,b]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a+1,b] = w_h[a+1,b] +0.1*w_h[a,b]      
        if z[a,b]-z[a+1,b-1]>20:
          w_h[a+1,b-1]=0.9*w_h[a,b]
          w_h[a+1,b-1] = w_h[a+1,b-1] +0.1*w_h[a,b]
        if z[a,b]-z[a+1,b+1]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a+1,b+1] = w_h[a+1,b+1] +0.1*w_h[a,b]
        if z[a,b]-z[a-1,b-1]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a-1,b-1] = w_h[a-1,b-1] +0.1*w_h[a,b]
        if z[a,b]-z[a-1,b]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a-1,b] = w_h[a-1,b] +0.1*w_h[a,b]
        if z[a,b]-z[a-1,b+1]>20:
          w_h[a,b]=0.9*w_h[a,b]
          w_h[a-1,b+1] = w_h[a-1,b+1] +0.1*w_h[a,b]
        
        z = dem + w_h
    return w_h, z


In [ ]:
from tqdm import tqdm
def flood(r, dt, number, precip, dem, Cd, cb):
  h0 = precip[0]
  w_h = np.full((len(dem), len(dem[0])), h0*dt)
  z = dem+w_h
  z_record = []
  w_h_record =[]
  giving_water = np.zeros((len(dem), len(dem[0])))
  precip = np.full((len(dem), len(dem[0])), h0*dt)
  c_V = np.zeros((len(dem),len(dem[0])))
  for i in tqdm(range(0, number)):
    h0 = precip[i+1]*dt
    (d, d_i) = direction(z)
    u = vel (r, d_i, Cd, d, w_h, dem, h0)
    z, w_h, c_V = vol(z, r, d_i, dem,w_h,c_V, u, d, cb, h0, dt)
   # w_h, z = balance(z, w_h, dem)
    z_record.append(z)
    w_h_record.append(w_h)
    w_h = w_h + h0
    z = dem + w_h
  
  return z_record, w_h_record, c_V
    

In [ ]:

(z_record, w_h_record, c_V) = flood(1, 105, 10, ida_m, dem, Cd, cb)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
z_record[7]

In [ ]:
w_h_record[6]

In [ ]:
w_h_record[4][1000,1200:1500]

In [ ]:
max(w_h_record[6][580])

0.007

In [ ]:
import gdal

In [ ]:
dst_filename = 'wh_record(0).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[0])


0

In [ ]:
dst_filename = 'wh_record(1)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[1])


0

In [ ]:
dst_filename = 'wh_record(2)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[2])


0

In [ ]:
dst_filename = 'wh_record(3)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[3])


0

In [ ]:
dst_filename = 'wh_record(4)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[4])

0

In [ ]:
dst_filename = 'wh_record(7)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[7])


0

In [ ]:
dst_filename = 'wh_record(8)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[8])


0

In [ ]:
dst_filename = 'wh_record(9)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[9])


0

In [ ]:
dst_filename = 'w_h_record(4)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[4])


0

In [ ]:
dst_filename = 'wh_record(5)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[5])


0

In [ ]:
dst_filename = 'wh_record(6)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[6])


0

In [ ]:
dst_filename = 'z_record(7)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[7])


0

In [ ]:
dst_filename = 'z_record(8)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[8])


0

In [ ]:
dst_filename = 'c_V.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(c_V)


0

In [ ]:
c_V.max()

50.27425816516227

In [ ]:
dst_filename = 'z_record(9).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(z_record[9])


0

In [ ]:
dst_filename = 'z_record(4).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(z_record[4])


0

In [ ]:
dst_filename = 'z_record(6).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(z_record[6])

0

In [ ]:
dst_filename = 'dem.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(dem)

0

In [ ]:

dst_filename = '10*6.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(w_h_3)


NameError: ignored

In [ ]:
w_h_3 = np.array(w_h_record[3])

In [ ]:
w_h_3 

array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       ...,
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 0., 0., ..., 0., 0., 0.],
       [2., 2., 2., ..., 2., 2., 2.]])

In [ ]:
dst_filename = '8w_h_record(4).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float64)
  
dataset.GetRasterBand(1).WriteArray(w_h_record[4])

0

In [ ]:
dst_filename = '7z_record(5).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[5])

IndexError: ignored

In [ ]:
dem[450]

array([18.85, 18.91, 18.85, ..., 42.83, 42.79, 42.76])

In [ ]:

dst_filename = 'dem.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(dem)

# follow code is adding GeoTranform and Projection

  

0

In [ ]:
dst_filename = 'z_record(2)__.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[2])

0

In [ ]:
dst_filename = 'z_record(3).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[3])


0

In [ ]:
dst_filename = 'z_record(4).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[4])


0

In [ ]:
dst_filename = 'z_record(5).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[5])


0

In [ ]:
dst_filename = 'z_record(6).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[6])


0

In [ ]:
dst_filename = 'z_record(7).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[7])


0

In [ ]:

g = gdal.Open ( "st.tif" )
# follow code is adding GeoTranform and Projection
geotrans=g.GetGeoTransform()  #get GeoTranform from existed 'data0'
proj=g.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None
  

AttributeError: ignored

In [ ]:
dst_filename = 'z_record(17).tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
  
dataset.GetRasterBand(1).WriteArray(z_record[17])
g = gdal.Open ( "st.tif" )
# follow code is adding GeoTranform and Projection
geotrans=g.GetGeoTransform()  #get GeoTranform from existed 'data0'
proj=g.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None
  

In [ ]:
    dst_filename = str(i)+'_'+str(z_record)+ '.tiff'
    x_pixels = len(dem)  # number of pixels in x
    y_pixels = len(dem[0])  # number of pixels in y
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
    h=z_record[0]
    dataset.GetRasterBand(1).WriteArray(h)

AttributeError: ignored

In [ ]:
save(z_record)

NameError: ignored

In [ ]:
dst_filename = 'h_50.tiff'
x_pixels = len(dem)  # number of pixels in x
y_pixels = len(dem[0])  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(z_record[49])

g = gdal.Open ( "st.tif" )
# follow code is adding GeoTranform and Projection
geotrans=g.GetGeoTransform()  #get GeoTranform from existed 'data0'
proj=g.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [ ]:
r =1
dt = 30
number = 3
precip= 0.00001



In [ ]:
  h0 = precip
  z_record = []
  w_h_record =[]
  (d, d_i) = direction(dem)
  u= vel_init(r, d_i, dem, Cd, d, h0)


In [ ]:
h_3


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.71240000e+01, 1.12496443e+07, ...,
        2.30240000e+01, 3.36879972e+08, 2.31420000e+01],
       [0.00000000e+00, 1.29812336e+07, 2.70640000e+01, ...,
        8.03858676e+08, 2.29640000e+01, 2.30920000e+01],
       ...,
       [0.00000000e+00, 1.13524604e+06, 1.79640000e+01, ...,
        1.28608064e+10, 3.10340000e+01, 3.10580000e+01],
       [0.00000000e+00, 1.79140000e+01, 2.19573154e+07, ...,
        1.14495078e+06, 1.66314617e+03, 3.11220000e+01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
    (d, d_i) = direction(z)


In [ ]:
    u = vel(r, d_i, z, Cd, d, w_h_1, dem)



In [ ]:
    z, w_h = vol(d_i, z, u, w_h_1, d, cb, h0, dt, c_V)

In [ ]:
    z_record.append(z)
    w_h_record.append(w_h)

In [ ]:
z

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.71120000e+01, 1.47448600e+03, ...,
        2.30120000e+01, 2.52813096e+04, 2.31060000e+01],
       [0.00000000e+00, 1.03563198e+03, 2.70520000e+01, ...,
        3.97712093e+05, 2.29520000e+01, 2.30560000e+01],
       ...,
       [0.00000000e+00, 1.95913410e+02, 2.01988026e+01, ...,
        3.14890597e+01, 3.16607705e+01, 3.10349966e+01],
       [0.00000000e+00, 1.79020000e+01, 1.62387916e+03, ...,
        2.16252252e+02, 8.12708215e+01, 3.10860000e+01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])